# Utils

In [109]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [83]:
#! pip install cufflinks
#! pip install plotly
import plotly as plotly
import plotly.plotly as py
import cufflinks as cf
plotly.tools.set_credentials_file(username='nhuang37', api_key='mPgr4qxmwZI8zt5h4JCR')

# Import data

In [5]:
cwd = os.getcwd()
cwd

'/Users/nhuang37/Desktop/NYU DS/job search/IBM/Data Challenge'

In [6]:
file = 'IBM_CAO_data_challenge_2018fall.xlsx'
xl = pd.ExcelFile(file)

In [17]:
# Load a sheet into a DataFrame by name
df = xl.parse('Saasy Business LLC transactions')

In [10]:
df.head()

,DATE,CLIENT_ID,CLIENT_SIZE,INDUSTRY,STATE,PRODUCT,NUM_LICENSE,PRICE_PER_LICENSE,NPS
0,2014-01-02,9761603,500+,IT,NJ,A,26,770.0,6.0
1,2014-01-02,4856499,500+,HEALTH CARE,TX,A,15,930.0,7.0
2,2014-01-02,5855785,0-19,IT,MN,A,1,980.0,5.0
3,2014-01-02,4095393,20-99,IT,NJ,B,4,500.0,9.0
4,2014-01-02,8152678,0-19,IT,PA,D,3,768.0,6.0


In [13]:
df.describe(include='all')

,DATE,CLIENT_ID,CLIENT_SIZE,INDUSTRY,STATE,PRODUCT,NUM_LICENSE,PRICE_PER_LICENSE,NPS
count,21239,2.123900e+04,21239,21239,21239,21239,21239.000000,21239.000000,20158.000000
unique,1276,NaN,4,5,16,5,NaN,NaN,NaN
top,2016-01-08 00:00:00,NaN,0-19,IT,CA,A,NaN,NaN,NaN
freq,44,NaN,9326,7612,3196,8684,NaN,NaN,NaN
first,2014-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2017-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,5.495317e+06,NaN,NaN,NaN,NaN,6.945431,746.339696,6.377121
std,NaN,2.581187e+06,NaN,NaN,NaN,NaN,6.341194,229.182367,1.972263
min,NaN,1.000810e+06,NaN,NaN,NaN,NaN,1.000000,142.000000,1.000000
25%,NaN,3.246700e+06,NaN,NaN,NaN,NaN,3.000000,576.000000,5.000000


# Calculate revenue growth
Note: monthly sales growth exhibits strong seasonal effect because the license is per annual term

In [86]:
#calculate revenue = NUM_LICENSE * PRICE_PER_LICENSE
#monthly revenue: create month-year variable, group by it & product, and obtatin the monthly sum per each product
#calculate monthly revenue growth per product in new dataframe, average it to get the highest monthly rev growth one
df['REVENUE'] = df['NUM_LICENSE'] * df['PRICE_PER_LICENSE']
#df.head(300)

In [25]:
# create month-year variable
df.index=pd.to_datetime(df['DATE'],format='%Y-%m-%d')

In [31]:
# get unique product names
product = df['PRODUCT'].unique().tolist()

In [126]:
#loop through each product category to calculate their annual sales sum
def revenue(freq, periods): #freq: 'Y' or 'M'; periods: '1' or '12'
    rev = {}
    for key in product:
        dfsub = df.loc[df.PRODUCT == key] #subset each product category
        rev[key] = dfsub.REVENUE.groupby(pd.Grouper(freq=freq)).sum() #calculate annual revenue per product

    result = pd.DataFrame(rev)
    growth = result.pct_change(periods=periods)
    return result, growth


In [128]:
Annual_result, Annual_growth = revenue('Y',1)
Monthly_result, Monthly_growth = revenue('M',1)
Monthly_yoy_result, Monthly_yoy_growth = revenue('M',12)

In [105]:
#sanity check with the total sum to ensure the calculation is correct
if Annual_result.sum(axis=1).all() == df.REVENUE.groupby(pd.Grouper(freq='Y')).sum().all():
    print ("passed!")

passed!


# Annual result

In [122]:
Annual_result

,A,B,C,D,E
DATE,,,,,
2014-12-31,10257228.0,1434632.0,723168.0,5083144.0,NaN
2015-12-31,14962020.0,2321082.0,1613052.0,8103099.2,NaN
2016-12-31,15706218.0,2915678.0,2634639.6,9762222.4,32259.6
2017-12-31,14235042.0,3301752.0,4020631.2,8811371.2,1390624.8


In [108]:
Annual_growth

,A,B,C,D,E
DATE,,,,,
2014-12-31,NaN,NaN,NaN,NaN,NaN
2015-12-31,0.458681,0.617894,1.230536,0.594112,NaN
2016-12-31,0.049739,0.256172,0.633326,0.204752,NaN
2017-12-31,-0.093668,0.132413,0.526065,-0.097401,42.107317


In [124]:
#visualization
cf.set_config_file(offline=False, world_readable=True, theme='ggplot')
Annual_growth.drop(['E'], axis=1).iplot(kind='scatter')

### Result:
- Annual growth is not suitable to analyze product E, as it was only introduced to the market for a year or so
- Among the other 4 products, product C is the strongest

# Monthly result

In [112]:
#see the average revenue growth
Monthly_growth.describe()

,A,B,C,D,E
count,47.000000,47.000000,47.000000,47.000000,15.000000
mean,0.009850,0.033691,0.122202,0.014068,0.488977
std,0.134440,0.186830,0.540657,0.110330,0.534615
min,-0.139551,-0.272936,-0.605025,-0.152021,0.011027
25%,-0.063449,-0.111395,-0.097441,-0.045241,0.177931
50%,-0.005521,0.002396,0.021427,-0.018179,0.265067
75%,0.047596,0.115691,0.262820,0.053107,0.554836
max,0.726833,0.585647,3.305043,0.385179,1.974963


In [136]:
#Visualization

cf.set_config_file(offline=False, world_readable=True, theme='pearl')
Monthly_growth.iplot(subplots=True, shape=(5,1), shared_xaxes=True, fill=True)

### Result
- Product E has the highest average monthly growth (~49%), but it only has 15 monthly revenue growth data, as it was introduced to market later
- Excluding Product E, Product C performs the best among the group, with highest average monthly growth (12.2%)
- All producst has high growth rate in Janaurary (strong time series cycling effect)

# Monthly YOY result


In [129]:
#see the average revenue growth
Monthly_yoy_growth.describe()

,A,B,C,D,E
count,36.000000,36.000000,36.000000,36.000000,4.000000
mean,0.138893,0.339917,0.898846,0.236181,50.602664
std,0.247270,0.242254,0.612762,0.304102,50.634900
min,-0.192355,-0.039592,0.176862,-0.255341,14.147412
25%,-0.066339,0.189489,0.524291,-0.012446,20.075035
50%,0.053367,0.276507,0.676754,0.203468,31.858573
75%,0.385938,0.524234,1.163944,0.518907,62.386203
max,0.715430,1.017940,2.765211,0.771107,124.546097


In [134]:
#Visualization

cf.set_config_file(offline=False, world_readable=True, theme='pearl')
Monthly_yoy_growth.iplot(subplots=True, shape=(5,1), shared_xaxes=True, fill=True)

# Determine the major factor
- Decision tree/Random Forest to see which factor contributes the most
- Machine learning approach to obtain the optimal weights in explaining the growth

## Descriptive analysis: 
- client records
- frequency table

In [147]:
#NOTE that there are clients signed multiple contracts per year
df['CLIENT_ID'].value_counts()

9491608    12
7836400    11
6036881    11
1571030    11
3766702    11
8137473    10
2914912    10
6315499    10
1244631    10
1841681    10
9024753    10
7136988    10
2857397    10
8290146    10
5652023     9
6200690     9
3382037     9
4058443     9
7078449     9
3824616     9
3236339     9
8307285     9
8241068     9
4247612     9
3415515     9
5422257     9
9880132     9
9519009     9
1220564     9
6987328     9
           ..
8217030     1
5761969     1
1482117     1
8269155     1
7636246     1
3547061     1
5629194     1
1037564     1
7906546     1
4944671     1
1127584     1
1870983     1
2897113     1
2098742     1
9956955     1
6334050     1
7054962     1
5439129     1
4656807     1
8920209     1
7931622     1
5144377     1
5832521     1
9793355     1
6506350     1
9080695     1
7447443     1
1707911     1
7496966     1
3883232     1
Name: CLIENT_ID, Length: 7499, dtype: int64

In [150]:
#example client
df.loc[df.CLIENT_ID == 4247612] 

,CLIENT_ID,CLIENT_SIZE,INDUSTRY,STATE,PRODUCT,NUM_LICENSE,PRICE_PER_LICENSE,NPS,REVENUE
DATE,,,,,,,,,
2014-07-23,4247612,0-19,REAL ESTATE,NY,A,2,990.0,2.0,1980.0
2016-01-08,4247612,0-19,REAL ESTATE,NY,A,2,990.0,2.0,1980.0
2016-05-24,4247612,0-19,REAL ESTATE,NY,A,3,970.0,2.0,2910.0
2016-06-15,4247612,0-19,REAL ESTATE,NY,A,4,1000.0,5.0,4000.0
2016-08-22,4247612,0-19,REAL ESTATE,NY,A,1,980.0,7.0,980.0
2016-10-28,4247612,0-19,REAL ESTATE,NY,D,4,776.0,2.0,3104.0
2016-11-14,4247612,0-19,REAL ESTATE,NY,D,4,768.0,2.0,3072.0
2016-11-22,4247612,0-19,REAL ESTATE,NY,A,1,1000.0,3.0,1000.0
2017-01-08,4247612,0-19,REAL ESTATE,NY,A,2,1000.0,7.0,2000.0


In [151]:
#descriptive statistics - two way table for categorical

# Table of product vs. industry
product_industry = pd.crosstab(index=df["INDUSTRY"], 
                           columns=df["PRODUCT"], margins=True)
product_industry

PRODUCT,A,B,C,D,E,All
INDUSTRY,,,,,,
FINANCE,2328,805,506,1531,274,5444
HEALTH CARE,1181,378,242,754,161,2716
IT,3022,1243,766,2069,512,7612
REAL ESTATE,1002,470,253,799,169,2693
RETAIL,1151,404,264,766,189,2774
All,8684,3300,2031,5919,1305,21239


In [155]:
product_industry/product_industry.loc["All"]    # Divide by column totals

PRODUCT,A,B,C,D,E,All
INDUSTRY,,,,,,
FINANCE,0.268079,0.243939,0.249138,0.258659,0.209962,0.256321
HEALTH CARE,0.135997,0.114545,0.119153,0.127386,0.123372,0.127878
IT,0.347996,0.376667,0.377154,0.349552,0.392337,0.358397
REAL ESTATE,0.115385,0.142424,0.124569,0.134989,0.129502,0.126795
RETAIL,0.132543,0.122424,0.129985,0.129414,0.144828,0.130609
All,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [159]:
# Table of product vs. client size
product_size = pd.crosstab(index=df["CLIENT_SIZE"], 
                           columns=df["PRODUCT"], margins=True)
product_size
product_size/product_size.loc["All"]    # Divide by column totals

PRODUCT,A,B,C,D,E,All
CLIENT_SIZE,,,,,,
0-19,0.429756,0.441212,0.450517,0.434702,0.498084,0.439098
100-499,0.207163,0.214545,0.209257,0.200203,0.171648,0.204388
20-99,0.249309,0.247879,0.241753,0.264234,0.243678,0.252178
500+,0.113772,0.096364,0.098474,0.100862,0.086590,0.104336
All,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
#descriptive stat - correlation for continuous

- presentation: stacked bar chart (x-axis: time stamp; y-axis: revenue growth)
- presentation: geo-graphical map!